In [11]:
import pandas as pd
pd.options.mode.copy_on_write = True
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import itertools
from tqdm import tqdm

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 00. Init

In [12]:
root_dir = '../'

cfg.init(filename=f"{root_dir}/config.yaml")

input_dir = f"{cfg.config['out_dir']}/out-publish-normalized"

In [13]:
COMBINE_harmonizer.init_redcap(token=cfg.config["redcap_token"], host=cfg.config["redcap_host"])

## 01. Clear Records

In [14]:
params = {
    'content': 'record',
    'fields': ['study_unique_id'],
}

err, redcap_data = COMBINE_harmonizer.get_redcap_data(params)

len(redcap_data)

532

In [15]:
if len(redcap_data) > 0:
    df = pd.DataFrame(redcap_data)

    df_groupby = df.groupby(['study_unique_id']).agg(count=('study_unique_id', 'count'))
    is_invalid = df_groupby['count'] > 1
    df_groupby[is_invalid]

In [16]:
if len(redcap_data) > 0:
    study_unique_ids = list(df['study_unique_id'].unique())
else:
    study_unique_ids = []

In [17]:
for study_unique_id_chunk in tqdm(study_unique_ids):
    COMBINE_harmonizer.delete_redcap_data([study_unique_id_chunk])

100%|██████████| 532/532 [02:26<00:00,  3.64it/s]


## 02. Clear Metadata

In [18]:
COMBINE_harmonizer.put_redcap_data('metadata', [])

(None, 0)

## 03. Clear Events

In [19]:
err, events = COMBINE_harmonizer.get_redcap_data({'content': 'event'})
event_ids = [each['unique_event_name'] for each in events]

len(event_ids)

6

In [20]:
if len(event_ids) > 0:
    COMBINE_harmonizer.delete_redcap_data(event_ids, content='event')